In [1]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/MyDrive/Colab\ Notebooks/ExpDesign

Mounted at /content/drive/
/content/drive/MyDrive/Colab Notebooks/ExpDesign


In [2]:
!pip install rdflib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.9/564.9 kB 9.5 MB/s eta 0:00:00


In [3]:
import os
import numpy as np
import pandas as pd
import pyarrow
import requests
from rdflib import Graph

# Set random seed for reproducibility
SEED = 42
np.random.seed(SEED)

# Configurations
DATA_DIR = "./data"
MODEL_DIR = "./models"
RESULTS_DIR = "./results"
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

In [ ]:
import sklearn
sklearn.__version__

'1.6.1'

In [ ]:
pd.__version__

'2.2.2'

## OHSUMED Articles data
The data is available on hugging face community datasets ([link](https://huggingface.co/datasets/community-datasets/ohsumed/viewer/ohsumed)), however does not contain the MESH labels for categorization or the relevance from querying. This data contains the mesh terms associated with the text, the title and abstract (which are used in the paper as the basis for prediction), publication type, author and source. Because of the lack of labels from the original data, this information is extracted from other sources as explained in the next sections.

In [4]:
train = pd.read_parquet('./data/raw/ohsumed/train-00000-of-00001.parquet')
test = pd.read_parquet('./data/raw/ohsumed/test-00000-of-00001.parquet')

df = pd.concat([train, test])
df.head()

,seq_id,medline_ui,mesh_terms,title,publication_type,abstract,author,source
0,1,87049087,Allied Health Personnel/*; Electric Countersho...,Refibrillation managed by EMT-Ds: incidence an...,JOURNAL ARTICLE.,Some patients converted from ventricular fibri...,Stults KR; Brown DD.,Am J Emerg Med 8703; 4(6):491-5
1,1,87049088,"Antidepressive Agents, Tricyclic/*PO; Arrhythm...",Tricyclic antidepressant overdose: emergency d...,JOURNAL ARTICLE.,There is controversy regarding the appropriate...,Foulke GE; Albertson TE; Walby WF.,Am J Emerg Med 8703; 4(6):496-500
2,1,87049089,Adult; Aircraft/*; Altitude/*; Blood Gas Monit...,Transconjunctival oxygen monitoring as a predi...,JOURNAL ARTICLE.,As the use of helicopters for air transport of...,Shufflebarger C; Jehle D; Cottington E; Martin M.,Am J Emerg Med 8703; 4(6):501-3
3,1,87049090,Adolescence; Adult; Aged; Blood Glucose/*ME; D...,Serum glucose changes after administration of ...,JOURNAL ARTICLE.,A prospective clinical trial was conducted to ...,Adler PM.,Am J Emerg Med 8703; 4(6):504-6
4,1,87049092,"Aged; Aged, 80 and over; Case Report; Female; ...",Nasogastric intubation: morbidity in an asympt...,JOURNAL ARTICLE.,An unusual case of a misdirected nasogastric t...,Gough D; Rust D.,Am J Emerg Med 8703; 4(6):511-3


## Creating sensitivity labels

The MESH labels used in the paper are said to be C12 and C13, which can be found on the following site: [link](https://meshb-prev.nlm.nih.gov/treeView). The 2025 corresponding labels for these categories are C12.050 for Female Urogenital Diseases and Pregnancy Complications, and C12.200 for Male Urogenital Diseases. There are several way to obtain this data, such as MESH API or direct RDF querying (see: [link](https://id.nlm.nih.gov/mesh/)), however due to errors, here the label terms are extracted as plain text and then filtered using python code.

According to the paper on this dataset specifically done by the same authors and referenced in the paper we are trying to reproduce ([link](https://doi.org/10.1145/3331184.3331256)), if any of the MESH terms are found within the MESH terms column, that article is considered to be sensitive.

In [5]:
import re

with open('./data/raw/ohsumed/c12.txt', 'r') as file:
    lines = file.readlines()

pattern = r'\[.*?\]'

c12 = [re.sub(pattern, '', line).strip() for line in lines]
c12 = [line for line in c12 if line != '']

print(c12)

['Female Urogenital Diseases and Pregnancy Complications', 'Female Urogenital Diseases', 'Genital Diseases, Female', 'Adnexal Diseases', 'Fallopian Tube Diseases', 'Fallopian Tube Neoplasms', 'Salpingitis', 'Ovarian Diseases', 'Anovulation', 'Menopause, Premature', 'Oophoritis', 'Ovarian Cysts', 'Polycystic Ovary Syndrome', 'Ovarian Hyperstimulation Syndrome', 'Ovarian Neoplasms', 'Brenner Tumor', 'Carcinoma, Endometrioid', 'Carcinoma, Ovarian Epithelial', 'Granulosa Cell Tumor', 'Hereditary Breast and Ovarian Cancer Syndrome', 'Luteoma', 'Meigs Syndrome', 'Sertoli-Leydig Cell Tumor', 'Thecoma', 'Ovarian Torsion', 'Primary Ovarian Insufficiency', 'Pelvic Inflammatory Disease', 'Endometritis', 'Oophoritis', 'Parametritis', 'Salpingitis', 'Dyspareunia', 'Endometriosis', 'Genital Neoplasms, Female', 'Gynatresia', 'Herpes Genitalis', 'Infertility, Female', 'Reproductive Tract Infections', 'Tuberculosis, Female Genital', 'Uterine Diseases', 'Adenomyosis', 'Endometrial Hyperplasia', 'Endomet

In [6]:
with open('./data/raw/ohsumed/c13.txt', 'r') as file:
    lines = file.readlines()

pattern = r'\[.*?\]'

c13 = [re.sub(pattern, '', line).strip() for line in lines]
c13 = [line for line in c13 if line != '']

print(c13)

['Male Urogenital Diseases', 'Genital Diseases, Male', 'Dyspareunia', 'Ejaculatory Dysfunction', 'Premature Ejaculation', 'Retrograde Ejaculation', 'Epididymitis', 'Erectile Dysfunction', 'Impotence, Vasculogenic', 'Fournier Gangrene', 'Genital Neoplasms, Male', 'Penile Neoplasms', 'Prostatic Neoplasms', 'Prostatic Neoplasms, Castration-Resistant', 'Testicular Neoplasms', 'Sertoli-Leydig Cell Tumor', 'Leydig Cell Tumor', 'Sertoli Cell Tumor', 'Hematocele', 'Hemospermia', 'Herpes Genitalis', 'Infertility, Male', 'Aspermia', 'Asthenozoospermia', 'Azoospermia', 'Oligospermia', 'Sertoli Cell-Only Syndrome', 'Teratozoospermia', 'Penile Diseases', 'Balanitis', 'Balanitis Xerotica Obliterans', 'Hypospadias', 'Penile Induration', 'Penile Neoplasms', 'Phimosis', 'Paraphimosis', 'Priapism', 'Prostatic Diseases', 'Prostatic Hyperplasia', 'Prostatic Neoplasms', 'Prostatic Neoplasms, Castration-Resistant', 'Prostatitis', 'Reproductive Tract Infections', 'Spermatic Cord Torsion', 'Spermatocele', 'Te

In [7]:
terms = c12 + c13

pattern = '|'.join([re.escape(term) for term in terms])

df['sensitive'] = df['mesh_terms'].str.contains(pattern, case=False, na=False).astype(int)

#df.to_csv('../data/raw/ohsumed/ohsumed.csv', index=False)

In [ ]:
df.head(10)

,seq_id,medline_ui,mesh_terms,title,publication_type,abstract,author,source
0,1,87049087,Allied Health Personnel/*; Electric Countersho...,Refibrillation managed by EMT-Ds: incidence an...,JOURNAL ARTICLE.,Some patients converted from ventricular fibri...,Stults KR; Brown DD.,Am J Emerg Med 8703; 4(6):491-5
1,1,87049088,"Antidepressive Agents, Tricyclic/*PO; Arrhythm...",Tricyclic antidepressant overdose: emergency d...,JOURNAL ARTICLE.,There is controversy regarding the appropriate...,Foulke GE; Albertson TE; Walby WF.,Am J Emerg Med 8703; 4(6):496-500
2,1,87049089,Adult; Aircraft/*; Altitude/*; Blood Gas Monit...,Transconjunctival oxygen monitoring as a predi...,JOURNAL ARTICLE.,As the use of helicopters for air transport of...,Shufflebarger C; Jehle D; Cottington E; Martin M.,Am J Emerg Med 8703; 4(6):501-3
3,1,87049090,Adolescence; Adult; Aged; Blood Glucose/*ME; D...,Serum glucose changes after administration of ...,JOURNAL ARTICLE.,A prospective clinical trial was conducted to ...,Adler PM.,Am J Emerg Med 8703; 4(6):504-6
4,1,87049092,"Aged; Aged, 80 and over; Case Report; Female; ...",Nasogastric intubation: morbidity in an asympt...,JOURNAL ARTICLE.,An unusual case of a misdirected nasogastric t...,Gough D; Rust D.,Am J Emerg Med 8703; 4(6):511-3
5,1,87049093,"Abdominal Injuries/ET; Accidents, Occupational...",Massive transfusion without major complication...,JOURNAL ARTICLE.,A case of massive degloving injury of the trun...,Brotman S; Lamonica C; Cowley RA.,Am J Emerg Med 8703; 4(6):514-5
6,1,87049094,Adult; Carbon Monoxide Poisoning/CO/*TH; Femal...,Acute carbon monoxide poisoning during pregnancy.,JOURNAL ARTICLE.,The course of a pregnant patient at term who w...,Margulies JL.,Am J Emerg Med 8703; 4(6):516-9
7,1,87049096,"Case Report; Child, Preschool; Drug Administra...",Intraosseous infusion of phenytoin.,JOURNAL ARTICLE.,"In the critically ill child, administration of...",Walsh-Kelly CM; Berens RJ; Glaeser PW; Losek JD.,Am J Emerg Med 8703; 4(6):523-4
8,1,87049098,Alcohol Drinking; Case Report; Emergencies; Es...,Boerhaave's syndrome: an elusive diagnosis.,JOURNAL ARTICLE.,Boerhaave's syndrome represents a diagnostic d...,Schwartz JA; Turnbull TL; Dymowski J; Uehara DT.,Am J Emerg Med 8703; 4(6):532-6
9,1,87049099,Adult; Case Report; Electrocardiography/*; Ext...,Cases in electrocardiography.,JOURNAL ARTICLE.,Boerhaave's syndrome represents a diagnostic d...,Zimmers T.,Am J Emerg Med 8703; 4(6):537-9


## Add relevance annotations

For the relevance annotations, here is used the original repository containing the raw data ([link](https://dmice.ohsu.edu/hersh/ohsumed/)). which is also used within hugging face to make the parquet files used as base data. According to the aforementioned paper ([link](https://doi.org/10.1145/3331184.3331256)), there are 16140 total articles with relevance judgements. For that purpose, we are using the judged file from the repository to extract any judgements made within all iterations of experiments. Those articles that are not judged are left to have relevance None, which can be used to separate the train/valid and test splits.

In [8]:
judged = pd.read_csv('./data/raw/ohsumed/judged', sep='\t', header=None,
                     names=['query_id', 'document_ui', 'document_i', 'relevance1', 'relevance2', 'relevance3'])

judged

,query_id,document_ui,document_i,relevance1,relevance2,relevance3
0,1,87097544,40626,d,NaN,d
1,1,87153566,11852,n,NaN,n
2,1,87157536,12693,d,NaN,NaN
3,1,87157537,12694,d,NaN,NaN
4,1,87184723,15450,n,NaN,NaN
...,...,...,...,...,...,...
16135,106,91354564,337888,d,NaN,NaN
16136,106,91354570,348231,n,NaN,NaN
16137,106,91356830,338251,NaN,NaN,n
16138,106,91359739,338619,n,NaN,NaN


In [9]:
def check_relevance(row):
    values = row[['relevance1', 'relevance2', 'relevance3']].dropna().unique()
    return len(values) <= 1

judged['is_relevance_same'] = judged.apply(check_relevance, axis=1)

In [ ]:
judged[~judged['is_relevance_same']]

,query_id,document_ui,document_i,relevance1,relevance2,relevance3,is_relevance_same
209,2,89290673,145433,n,NaN,d,False
460,3,90355689,272646,n,NaN,p,False
551,4,88140817,66203,p,NaN,d,False
669,5,89045465,198897,n,NaN,d,False
875,7,90132815,216764,n,NaN,p,False
...,...,...,...,...,...,...,...
15965,106,88089024,107631,d,p,NaN,False
15970,106,88103602,108005,d,n,NaN,False
15978,106,88136867,109156,d,p,NaN,False
15982,106,88181136,83010,n,d,NaN,False


Since there are relevance judgements that are changed in different iterations, the judgements are done so that if there is a judgement during the first iteration, that is considered the relevance judgement for the row. If not, then revelance 2 is used, and if relevance 2 is also not available, then relevance 3 is used. This is deducted from the raw data readme file, however there is no explanation what judgements were used in the paper and how they were done. Because of that, the original judgement columns are also kept within the data for possible different experiments. Additionally, the column `relevance_majority` denotes the majority relevance judgements from all judgements if there is more than 1. In case there are only 2 different judgements, the earliest is considered valid.

In [10]:
judged = pd.read_csv('./data/raw/ohsumed/judged', sep='\t', header=None,
                     names=['query_id', 'document_ui', 'document_i', 'relevance1', 'relevance2', 'relevance3'])
judged

def get_final_relevance(row):
    if pd.notna(row['relevance1']):
        return row['relevance1']
    elif pd.notna(row['relevance2']):
        return row['relevance2']
    else:
        return row['relevance3'] if pd.notna(row['relevance3']) else 'not_judged'


def get_majority_relevance(row):
    judgments = [row['relevance1'], row['relevance2'], row['relevance3']]

    judgments = [j for j in judgments if pd.notna(j)]

    if len(judgments) == 1:
        return judgments[0]
    elif len(judgments) == 2 and judgments[0] == judgments[1]:
        return judgments[0]
    elif len(judgments) == 2 and judgments[0] != judgments[1]:
        return judgments[0]  # earliest judgment
    elif len(judgments) == 3:
        if judgments[0] == judgments[1] or judgments[0] == judgments[2]:
            return judgments[0]  # majority is the first judgment
        elif judgments[1] == judgments[2]:
            return judgments[1]
    elif len(judgments) == 3:
        return row['relevance1']  # earliest judgment

    return 'not_judged'


judged['final_relevance'] = judged.apply(get_final_relevance, axis=1)
judged['relevance_majority'] = judged.apply(get_majority_relevance, axis=1)
judged.head(10)

,query_id,document_ui,document_i,relevance1,relevance2,relevance3,final_relevance,relevance_majority
0,1,87097544,40626,d,NaN,d,d,d
1,1,87153566,11852,n,NaN,n,n,n
2,1,87157536,12693,d,NaN,NaN,d,d
3,1,87157537,12694,d,NaN,NaN,d,d
4,1,87184723,15450,n,NaN,NaN,n,n
5,1,87202778,17665,p,NaN,NaN,p,p
6,1,87210267,18432,n,NaN,NaN,n,n
7,1,87210456,18540,n,NaN,n,n,n
8,1,87210457,44695,n,NaN,NaN,n,n
9,1,87210458,18541,n,NaN,n,n,n


In [11]:
judgements = judged[['document_ui', 'relevance1', 'relevance2', 'relevance3', 'final_relevance', 'relevance_majority']]

df_merged = pd.merge(df, judgements, how='left', left_on='medline_ui', right_on='document_ui').drop(columns=['document_ui'])
#df_merged[~df_merged['relevance_status'].isna()]
df_merged['relevance_status'] = df_merged['final_relevance']
df_merged.head(15)

,seq_id,medline_ui,mesh_terms,title,publication_type,abstract,author,source,sensitive,relevance1,relevance2,relevance3,final_relevance,relevance_majority,relevance_status
0,1,87049087,Allied Health Personnel/*; Electric Countersho...,Refibrillation managed by EMT-Ds: incidence an...,JOURNAL ARTICLE.,Some patients converted from ventricular fibri...,Stults KR; Brown DD.,Am J Emerg Med 8703; 4(6):491-5,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,87049088,"Antidepressive Agents, Tricyclic/*PO; Arrhythm...",Tricyclic antidepressant overdose: emergency d...,JOURNAL ARTICLE.,There is controversy regarding the appropriate...,Foulke GE; Albertson TE; Walby WF.,Am J Emerg Med 8703; 4(6):496-500,0,NaN,NaN,NaN,NaN,NaN,NaN
2,1,87049089,Adult; Aircraft/*; Altitude/*; Blood Gas Monit...,Transconjunctival oxygen monitoring as a predi...,JOURNAL ARTICLE.,As the use of helicopters for air transport of...,Shufflebarger C; Jehle D; Cottington E; Martin M.,Am J Emerg Med 8703; 4(6):501-3,0,NaN,NaN,NaN,NaN,NaN,NaN
3,1,87049090,Adolescence; Adult; Aged; Blood Glucose/*ME; D...,Serum glucose changes after administration of ...,JOURNAL ARTICLE.,A prospective clinical trial was conducted to ...,Adler PM.,Am J Emerg Med 8703; 4(6):504-6,0,NaN,NaN,NaN,NaN,NaN,NaN
4,1,87049092,"Aged; Aged, 80 and over; Case Report; Female; ...",Nasogastric intubation: morbidity in an asympt...,JOURNAL ARTICLE.,An unusual case of a misdirected nasogastric t...,Gough D; Rust D.,Am J Emerg Med 8703; 4(6):511-3,0,NaN,NaN,NaN,NaN,NaN,NaN
5,1,87049093,"Abdominal Injuries/ET; Accidents, Occupational...",Massive transfusion without major complication...,JOURNAL ARTICLE.,A case of massive degloving injury of the trun...,Brotman S; Lamonica C; Cowley RA.,Am J Emerg Med 8703; 4(6):514-5,0,NaN,NaN,NaN,NaN,NaN,NaN
6,1,87049094,Adult; Carbon Monoxide Poisoning/CO/*TH; Femal...,Acute carbon monoxide poisoning during pregnancy.,JOURNAL ARTICLE.,The course of a pregnant patient at term who w...,Margulies JL.,Am J Emerg Med 8703; 4(6):516-9,1,NaN,NaN,NaN,NaN,NaN,NaN
7,1,87049096,"Case Report; Child, Preschool; Drug Administra...",Intraosseous infusion of phenytoin.,JOURNAL ARTICLE.,"In the critically ill child, administration of...",Walsh-Kelly CM; Berens RJ; Glaeser PW; Losek JD.,Am J Emerg Med 8703; 4(6):523-4,0,NaN,NaN,NaN,NaN,NaN,NaN
8,1,87049098,Alcohol Drinking; Case Report; Emergencies; Es...,Boerhaave's syndrome: an elusive diagnosis.,JOURNAL ARTICLE.,Boerhaave's syndrome represents a diagnostic d...,Schwartz JA; Turnbull TL; Dymowski J; Uehara DT.,Am J Emerg Med 8703; 4(6):532-6,0,NaN,NaN,NaN,NaN,NaN,NaN
9,1,87049099,Adult; Case Report; Electrocardiography/*; Ext...,Cases in electrocardiography.,JOURNAL ARTICLE.,Boerhaave's syndrome represents a diagnostic d...,Zimmers T.,Am J Emerg Med 8703; 4(6):537-9,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_merged[~df_merged['relevance_status'].isna()]

,seq_id,medline_ui,mesh_terms,title,publication_type,abstract,author,source,sensitive,relevance1,relevance2,relevance3,final_relevance,relevance_majority,relevance_status
125,1,87049457,"Adolescence; Anesthesia, Inhalation/*AE; Child...",Prospective study of liver function in childre...,JOURNAL ARTICLE.,"In the 4 years January 1981 to December 1984, ...",Wark H; O'Halloran M; Overton J.,Br J Anaesth 8703; 58(11):1224-8,0,NaN,NaN,n,n,n,n
153,1,87049559,Female; Human; Pregnancy; Propranolol/TU; Puer...,Postpartum thyroiditis--an underdiagnosed dise...,JOURNAL ARTICLE.,We present the case of an 11-month-old baby wi...,Ramsay I.,Br J Obstet Gynaecol 8703; 93(11):1121-3,1,n,n,NaN,n,n,n
222,1,87050621,Actinomycosis/*CO; Adult; Case Report; Human; ...,Primary renal actinomycosis in the presence of...,JOURNAL ARTICLE.,Twenty-five patients with fracture of the peni...,McGibney D; Clarke PB.,Br J Urol 8703; 58(5):566,1,n,NaN,n,n,n,n
282,1,87051243,"Adult; Aged; Antineoplastic Agents, Combined/*...",Clinical course of breast cancer patients with...,JOURNAL ARTICLE.,"Between July 1973 and December 1979, 1171 pati...",Scheid V; Buzdar AU; Smith TL; Hortobagyi GN.,Cancer 8703; 58(12):2589-93,0,d,NaN,NaN,d,d,d
299,1,87051269,Adult; Aged; Autopsy; Echocardiography; Electr...,Cardiac abnormalities in patients with diffuse...,JOURNAL ARTICLE.,Many patients with diffuse malignant pleural m...,Wadler S; Chahinian P; Slater W; Goldman M; Me...,Cancer 8703; 58(12):2744-50,0,NaN,NaN,n,n,n,n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350107,54711,91368351,Female; Human; Isoantibodies/AN; Pregnancy; Rh...,Controversies in transfusion medicine: Du test...,LETTER.,"A perfluorocarbon blood substitute, Fluosol, i...",Stroup M.,Transfusion 9112; 31(7):677-8,0,NaN,NaN,p,p,p,p
350205,54711,91374975,Antibodies/*AN; Antigenic Determinants; Blood ...,Significance of anti-phospholipid antibodies i...,JOURNAL ARTICLE.,Anti-phospholipid antibodies (APA) as markers ...,Frampton G; Hicks J; Cameron JS.,Kidney Int 9112; 39(6):1225-31,1,d,NaN,NaN,d,d,d
350206,54711,91374975,Antibodies/*AN; Antigenic Determinants; Blood ...,Significance of anti-phospholipid antibodies i...,JOURNAL ARTICLE.,Anti-phospholipid antibodies (APA) as markers ...,Frampton G; Hicks J; Cameron JS.,Kidney Int 9112; 39(6):1225-31,1,p,NaN,NaN,p,p,p
350240,54711,91376240,"Child Abuse; Child Day Care Centers/*; Child, ...",Requiring formal training in preventive health...,JOURNAL ARTICLE.,The study was a test of the feasibility of man...,Bassoff BZ; Willis WO.,Public Health Rep 9112; 106(5):523-9,0,n,NaN,NaN,n,n,n


## Next steps

The aforementioned paper ([link](https://doi.org/10.1145/3331184.3331256)) explains that the steps to preprocess the data for modelling include tf-idf transformation of the titles + abstracts of every article. I could not find any information whether they have removed any common words such as 'a', 'the' and so on.

Furthermore, in our paper, the data is separated in train, valid and test sets such that all of the articles containing both sensitivity and relevance annotations are used as test set (16140 in total), whereas from those without relevance annotations are used for train and validation with 85-15 split. The split between train/valid and test based on the relevance should be done with the relevance_status column (None means no relevance -> goes to train/valid split).

All of these steps should be done within the preprocessing of the data.

In [12]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import nltk

nltk.__version__

'3.9.1'

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
df_merged['text'] = df_merged['title'] + " " + df_merged['abstract']

In [15]:
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])
df_merged['text_cleaned'] = df_merged['text'].apply(remove_stopwords)

In [ ]:
df_merged.head()

,seq_id,medline_ui,mesh_terms,title,publication_type,abstract,author,source,sensitive,relevance1,relevance2,relevance3,final_relevance,relevance_majority,relevance_status,text,text_cleaned
0,1,87049087,Allied Health Personnel/*; Electric Countersho...,Refibrillation managed by EMT-Ds: incidence an...,JOURNAL ARTICLE.,Some patients converted from ventricular fibri...,Stults KR; Brown DD.,Am J Emerg Med 8703; 4(6):491-5,0,NaN,NaN,NaN,NaN,NaN,NaN,Refibrillation managed by EMT-Ds: incidence an...,Refibrillation managed EMT-Ds: incidence outco...
1,1,87049088,"Antidepressive Agents, Tricyclic/*PO; Arrhythm...",Tricyclic antidepressant overdose: emergency d...,JOURNAL ARTICLE.,There is controversy regarding the appropriate...,Foulke GE; Albertson TE; Walby WF.,Am J Emerg Med 8703; 4(6):496-500,0,NaN,NaN,NaN,NaN,NaN,NaN,Tricyclic antidepressant overdose: emergency d...,Tricyclic antidepressant overdose: emergency d...
2,1,87049089,Adult; Aircraft/*; Altitude/*; Blood Gas Monit...,Transconjunctival oxygen monitoring as a predi...,JOURNAL ARTICLE.,As the use of helicopters for air transport of...,Shufflebarger C; Jehle D; Cottington E; Martin M.,Am J Emerg Med 8703; 4(6):501-3,0,NaN,NaN,NaN,NaN,NaN,NaN,Transconjunctival oxygen monitoring as a predi...,Transconjunctival oxygen monitoring predictor ...
3,1,87049090,Adolescence; Adult; Aged; Blood Glucose/*ME; D...,Serum glucose changes after administration of ...,JOURNAL ARTICLE.,A prospective clinical trial was conducted to ...,Adler PM.,Am J Emerg Med 8703; 4(6):504-6,0,NaN,NaN,NaN,NaN,NaN,NaN,Serum glucose changes after administration of ...,Serum glucose changes administration 50% dextr...
4,1,87049092,"Aged; Aged, 80 and over; Case Report; Female; ...",Nasogastric intubation: morbidity in an asympt...,JOURNAL ARTICLE.,An unusual case of a misdirected nasogastric t...,Gough D; Rust D.,Am J Emerg Med 8703; 4(6):511-3,0,NaN,NaN,NaN,NaN,NaN,NaN,Nasogastric intubation: morbidity in an asympt...,Nasogastric intubation: morbidity asymptomatic...


Without limitation by use of *max_features*, total TF-IDF features are 164389

In [16]:
# Step 3: TF-IDF Transformation
tfidf_vectorizer = TfidfVectorizer(max_features=16000)  # Adjust max_features as needed
X_tfidf = tfidf_vectorizer.fit_transform(df_merged['text_cleaned'])

In [18]:
df_merged = df_merged.drop(labels=["title", "abstract", "text"], axis=1)

In [19]:
row_indices_known_relevance = df_merged['relevance_status'].isna()

test_set_y = df_merged[~row_indices_known_relevance]
test_set_X = X_tfidf[~row_indices_known_relevance]

train_valid_set_y = df_merged[row_indices_known_relevance]
train_valid_set_X = X_tfidf[row_indices_known_relevance]

In [20]:
train_set_y, valid_set_y, train_set_X, valid_set_X = train_test_split(train_valid_set_y, train_valid_set_X, test_size=0.15, random_state=42)

In [21]:
# Display sizes of the splits
print("Train Set X Size:", train_set_X.shape, "in sparse format", train_set_X.format)
print("Train Set y Size:", train_set_y.shape)
print("Validation Set X Size:", valid_set_X.shape, "in sparse format", valid_set_X.format)
print("Validation Set y Size:", valid_set_y.shape)
print("Test Set X Size:", test_set_X.shape, "in sparse format", test_set_X.format)
print("Test Set y Size:", test_set_y.shape)

Train Set X Size: (284013, 16000) in sparse format csr
Train Set y Size: (284013, 14)
Validation Set X Size: (50121, 16000) in sparse format csr
Validation Set y Size: (50121, 14)
Test Set X Size: (16140, 16000) in sparse format csr
Test Set y Size: (16140, 14)


In [22]:
# Step 5: Save the splits for modeling
train_set_y.to_csv('./data/processed/train_set.csv')
valid_set_y.to_csv('./data/processed/valid_set.csv')
test_set_y.to_csv('./data/processed/test_set.csv')

# Step 6: Save TF-IDF features (if required for later use)
import scipy.sparse
scipy.sparse.save_npz('./data/processed/tfidf_features_train.npz', train_set_X)
scipy.sparse.save_npz('./data/processed/tfidf_features_valid.npz', valid_set_X)
scipy.sparse.save_npz('./data/processed/tfidf_features_test.npz', test_set_X)


In [ ]:
# load like this:
#X_train = scipy.sparse.load_npz('./data/processed/tfidf_features_train.npz')

## Tests

In [ ]:
# load rdf
# g = Graph()
# g.parse("../data/raw/ohsumed/mesh/mesh2025.nt/filtered.nt", format="nt")

<Graph identifier=N9b144e94a7a0473780dba4f8dd538955 (<class 'rdflib.graph.Graph'>)>

In [ ]:
"""

query = "
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>

SELECT DISTINCT ?descriptor ?label ?treeNum
WHERE {

  ?descriptor meshv:treeNumber ?treeNum .
  ?descriptor rdfs:label ?label .

}
ORDER BY ?treeNum
"
# ?descriptor rdf:type meshv:TopicalDescriptor .
# FILTER (STRSTARTS(?treeNum, "C12.050") || STRSTARTS(?treeNum, "C12.200"))

results = g.query(query)
print(results)

for row in results:
    print(f"Tree Number: {row['treeNum']}, Label: {row['label']}")

    """

In [ ]:
"""
# Base URL for SPARQL
sparql_url = "https://id.nlm.nih.gov/mesh/sparql"

# SPARQL query to get all descriptors under C12.050
sparql_query = "
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
PREFIX mesh2025: <http://id.nlm.nih.gov/mesh/2025/>
PREFIX mesh2024: <http://id.nlm.nih.gov/mesh/2024/>
PREFIX mesh2023: <http://id.nlm.nih.gov/mesh/2023/>

SELECT DISTINCT ?descriptor ?label ?treeNum
WHERE {
  ?descriptor rdf:type meshv:TopicalDescriptor .
  ?descriptor meshv:treeNumber ?treeNum .
  ?descriptor rdfs:label ?label .
}
"

# FILTER (CONTAINS(?treeNum, "C12.050") || CONTAINS(?treeNum, "C12.200"))
# Function to query SPARQL endpoint
def query_sparql(query):
    params = {'query': query, 'format': 'application/json'}
    response = requests.get(sparql_url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed SPARQL query: {response.status_code}")
        return None

# Execute the query
sparql_data = query_sparql(sparql_query)

# Process and print results
if sparql_data:
    for result in sparql_data['results']['bindings']:
        descriptor = result['descriptor']['value']
        label = result['label']['value']
        print(f"Descriptor: {label} ({descriptor})")

"""

'\n# Base URL for SPARQL\nsparql_url = "https://id.nlm.nih.gov/mesh/sparql"\n\n# SPARQL query to get all descriptors under C12.050\nsparql_query = "\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\nPREFIX owl: <http://www.w3.org/2002/07/owl#>\nPREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>\nPREFIX mesh: <http://id.nlm.nih.gov/mesh/>\nPREFIX mesh2025: <http://id.nlm.nih.gov/mesh/2025/>\nPREFIX mesh2024: <http://id.nlm.nih.gov/mesh/2024/>\nPREFIX mesh2023: <http://id.nlm.nih.gov/mesh/2023/>\n\nSELECT DISTINCT ?descriptor ?label ?treeNum\nWHERE {\n  ?descriptor rdf:type meshv:TopicalDescriptor .\n  ?descriptor meshv:treeNumber ?treeNum .\n  ?descriptor rdfs:label ?label .\n}\n"\n\n# FILTER (CONTAINS(?treeNum, "C12.050") || CONTAINS(?treeNum, "C12.200"))\n# Function to query SPARQL endpoint\ndef query_sparql(query):\n    params = {\'query\': query, \'format\': \'application/